In [10]:
# use this to install benchpots in the colab env
# !pip install benchpots

In [11]:
import numpy as np
from benchpots.datasets import preprocess_physionet2012

In [12]:
features=['ALP', 
          'ALT', 
          'AST', 
          'Albumin',
          'BUN',
          'Bilirubin',
          'Cholesterol',
          'Creatinine',
          'FiO2',
          'GCS',
          'Glucose',
          'HCO3',
          'HCT',
          'HR', 
          'K', 
          'Lactate', 
          'Mg', 
          'Na', 
          'PaCO2', 
          'PaO2', 
          'Platelets', 
          'RespRate', 
          'SaO2', 
          'Temp', 
          'TroponinI', 
          'TroponinT', 
          'Urine', 
          'WBC', 
          'Weight', 
          'pH',

          'NISysABP',
          'SysABP',
          'NIMAP',
          'MAP',
          'NIDiasABP',
          'DiasABP',
          ]
data = preprocess_physionet2012('set-a', rate=0, features=features)

for key in ['train_X', 'val_X', 'test_X']:
    a, b = 35, 34
    idx = np.array(np.isnan(data[key][:, :,a]))
    data[key][:, :, a][idx] = data[key][:, :, b][idx]
    a, b = 33, 32
    idx = np.array(np.isnan(data[key][:, :,a]))
    data[key][:, :, a][idx] = data[key][:, :, b][idx]
    a, b = 31, 30
    idx = np.array(np.isnan(data[key][:, :,a]))
    data[key][:, :, a][idx] = data[key][:, :, b][idx]
    data[key] = data[key][:, :, [u for u in range(36) if u not in [30, 32, 34]]]

2024-10-25 16:40:50 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-10-25 16:40:50 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-10-25 16:40:50 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...


2024-10-25 16:40:50 [INFO]: Loaded successfully!
2024-10-25 16:40:56 [WARNING]: rate is 0, no missing values are artificially added.
2024-10-25 16:40:56 [INFO]: Total sample number: 3997
2024-10-25 16:40:56 [INFO]: Training set size: 2557 (63.97%)
2024-10-25 16:40:56 [INFO]: Validation set size: 640 (16.01%)
2024-10-25 16:40:56 [INFO]: Test set size: 800 (20.02%)
2024-10-25 16:40:56 [INFO]: Number of steps: 48
2024-10-25 16:40:56 [INFO]: Number of features: 36
2024-10-25 16:40:56 [INFO]: Train set missing rate: 79.51%
2024-10-25 16:40:56 [INFO]: Validating set missing rate: 79.63%
2024-10-25 16:40:56 [INFO]: Test set missing rate: 79.78%


In [13]:
for key in ['train_X', 'val_X', 'test_X']:
    for x, sample in enumerate(data[key]):
        replace = np.nanmean(sample, axis=0)
        nanidx = np.isnan(replace)
        for k, i in enumerate(replace):
            if nanidx[k]:
                i = 0
            data[key][x, :, k][np.array(np.isnan(sample[:, k]))] = i

/tmp/ipykernel_1346265/3113696779.py:3: RuntimeWarning: Mean of empty slice
  replace = np.nanmean(sample, axis=0)


In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, TensorDataset
from sklearn.metrics import roc_auc_score

class gruSimple(nn.Module):
    def __init__(
        self,
        n_features = 33,
        n_classes = 2,
        rnn_hidden_size = 43,
        num_layers = 1,
    ):
        super(gruSimple, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = rnn_hidden_size
        self.gru = nn.GRU(n_features, rnn_hidden_size, num_layers, batch_first=True)
        # Define the top regressor layer with batch normalization and dropout
        self.fc = nn.Linear(rnn_hidden_size, n_classes)
        self.batch_norm = nn.BatchNorm1d(n_classes)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Forward propagate through GRU
        out, _ = self.gru(x, h0)
        # Get the last hidden state
        last_hidden_state = out[:, -1, :]
        out = self.fc(last_hidden_state)
        out = self.batch_norm(out)
        out = self.dropout(out)
        out = F.softmax(out, dim=1)
        return out

In [22]:
# Early stopping parameters
early_stopping_patience = 3  # Number of epochs to wait for improvement
best_loss = float('inf')
patience_counter = 0

# Training function with early stopping
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)

    global best_loss, patience_counter
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for _, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(torch.nan_to_num(inputs))

            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/10:.4f}')
        running_loss = 0.0
        
        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(torch.nan_to_num(inputs))
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        print(f'Validation Loss after Epoch [{epoch+1}/{num_epochs}]: {val_loss:.4f}')
        
        # Early stopping check
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f'Early stopping at epoch {epoch+1}')
                break
        
        model.train()  # Set back to training mode

    print('Finished Training')

def evaluate_model(val_loader, model, device):
    model.to(device)
    model.eval()
    outputs = torch.tensor([]).to(device)
    lab = torch.tensor([]).to(device)
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            output = model(torch.nan_to_num(inputs))
            outputs = torch.concat([outputs, output])
            lab = torch.concat([lab, labels])
    return outputs, lab

In [16]:
traindata = TensorDataset(torch.tensor(data['train_X'].astype(np.float32)), torch.tensor(data['train_y']))
valdata = TensorDataset(torch.tensor(data['val_X'].astype(np.float32)), torch.tensor(data['val_y']))
testdata = TensorDataset(torch.tensor(data['test_X'].astype(np.float32)), torch.tensor(data['test_y']))

In [17]:
train_loader = DataLoader(traindata, batch_size=64)
val_loader = DataLoader(valdata, batch_size=64)
test_loader = DataLoader(testdata, batch_size=64)

In [23]:
model = gruSimple(33, 2, 64)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [25]:
# Train the model with early stopping
num_epochs = 50
train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

Epoch [1/50], Loss: 2.6781
Validation Loss after Epoch [1/50]: 0.6156
Epoch [2/50], Loss: 2.5417
Validation Loss after Epoch [2/50]: 0.6066
Epoch [3/50], Loss: 2.4729
Validation Loss after Epoch [3/50]: 0.5918
Epoch [4/50], Loss: 2.4135
Validation Loss after Epoch [4/50]: 0.5642
Epoch [5/50], Loss: 2.3176
Validation Loss after Epoch [5/50]: 0.5542
Epoch [6/50], Loss: 2.2747
Validation Loss after Epoch [6/50]: 0.5480
Epoch [7/50], Loss: 2.2535
Validation Loss after Epoch [7/50]: 0.5303
Epoch [8/50], Loss: 2.2495
Validation Loss after Epoch [8/50]: 0.5406
Epoch [9/50], Loss: 2.2274
Validation Loss after Epoch [9/50]: 0.5336
Epoch [10/50], Loss: 2.2156
Validation Loss after Epoch [10/50]: 0.5326
Early stopping at epoch 10
Finished Training


In [26]:
output, true = evaluate_model(test_loader, model, device)
display(roc_auc_score(true.cpu(), output[:, 1].cpu()))

0.7718066397311683